In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
"""
Create a storage for both stations we want to look into and anomaly
"""
storage_top = pd.DataFrame(columns=['STATION', 'DATE', 'TOTAL_TRAFFIC'])
storage_anomaly = pd.DataFrame(columns=['STATION', 'DATE', 'TOTAL_TRAFFIC'])

"""
Get list of turnstile url
"""
url = ['http://web.mta.info/developers/data/nyct/turnstile/turnstile_170304.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170311.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170318.txt', 
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170325.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170401.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170408.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt', 
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170422.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170429.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170506.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt', 
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170520.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170527.txt']

"""
Construct an interator that iterates through the weeks of interests
and allows for the analysis to run for a time of interest
Iterator needs to generate integer format 'yymmdd' for 2014-2017
"""

for i in tqdm(range(len(url))):

    # week = '170422' # 
    # datafile = 'turnstile_%s.txt' % week
    # url = 'http://web.mta.info/developers/data/nyct/turnstile/%s' % datafile

    """
    Create df object by reading url
    """


    df = pd.read_csv(url[i],header = 0,)
    ## Clean the header row
    df.rename(columns = {'EXITS                                                               ':'EXITS'}, inplace = True)
    
    """
    Create pivot table using pands to obtain top stations for each week
    """

    f = {'ENTRIES':['max','min'], 'EXITS':['max','min']}
    pivot = df.groupby(['STATION','UNIT','DATE','SCP'], as_index= False).agg(f)
    pivot['TOTAL_TRAFFIC'] = (pivot.ENTRIES['max'] - pivot.ENTRIES['min'])+(pivot.EXITS['max'] - pivot.EXITS['min'])
    pivot.columns = pivot.columns.droplevel(level=1)
    pivot.columns = (['STATION', 'UNIT', 'DATE', 'SCP', 'ENTRIES_MAX', 'ENTRIES_MIN', 'EXITS_MAX', 'EXITS_MIN', 'TOTAL_TRAFFIC'])

    pivot2 = pivot.groupby(['STATION','DATE'], as_index = False)[['TOTAL_TRAFFIC']].sum().sort_values('TOTAL_TRAFFIC', ascending =False)


    threshold = 1000000
    top = pivot2[pivot2['TOTAL_TRAFFIC']<threshold].reset_index().iloc[:30]
    anomaly = pivot2[pivot2['TOTAL_TRAFFIC']>threshold].reset_index()

    storage_top = storage_top.append(top,ignore_index = True).drop('index',1)
    storage_anomaly = storage_anomaly.append(anomaly,ignore_index = True).drop('index',1)










100%|██████████| 13/13 [01:13<00:00,  5.65s/it]


In [5]:
storage_anomaly

,DATE,STATION,TOTAL_TRAFFIC
0,03/01/2017,MORISN AV/SNDVW,1560333065
1,02/25/2017,SPRING ST,116737859
2,03/02/2017,AVENUE U,51068099
3,03/02/2017,FOREST HILLS 71,17553276
4,02/27/2017,VERNON-JACKSON,15361115
5,02/28/2017,MT EDEN AV,7256008
6,03/02/2017,SUTPHIN BLVD,3851406
7,03/03/2017,NEWARK BM BW,1074428
8,03/07/2017,75 AV,169578152
9,03/10/2017,45 ST,134278297
